# GPU Efficient LLM fine-tuning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//velebit-ai/research-llm-development/blob/master/GPU-efficient-LLM-fine-tuning.ipynb)

We'll go through an example of parameter efficinet GPU training on a T4 GPU by using Google Colab.

## Package setup

In [1]:
%pip install transformers -q
%pip install bitsandbytes -q
%pip install datasets -q
%pip install accelerate -q
%pip install peft -q
%pip install trl -q
%pip install einops -q
%pip install tensorboard -q

%pip install watermark -q # version checks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%load_ext watermark

## Loading the model

We will load the model by using the `transformers` library from Hugging Face.

In order to fit really large models into a single GPU, you can
load the model in half precision. Most LLMs are even trained in half precision (float16, bfloat16) and there is almost no performace loss compared with full precision (float32) training.

If that is not enough, you can quantize the weights of the model to 8bit or even 4bit. For that we need the `accelerate` and `bitsandbytes` library.

In [4]:
# Illustrate memory usage
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m").to("cuda")
model_fp16 = AutoModelForCausalLM.from_pretrained("facebook/opt-350m", torch_dtype="auto").to("cuda") # torch_dtype=torch.float16
model_8bit = AutoModelForCausalLM.from_pretrained("facebook/opt-350m", load_in_8bit=True) # 8bit and 4bit models are automatically loaded to GPU
model_4bit = AutoModelForCausalLM.from_pretrained("facebook/opt-350m", load_in_4bit=True)

def print_model_gpu_usage(model, name):
  params_size = sum([param.nelement() * param.element_size()
                    for param in model.parameters()])  # in bytes
  print(f"Model: {name} (GB): {params_size/1e9}")

print_model_gpu_usage(model, "base")
print_model_gpu_usage(model_fp16, "fp16")
print_model_gpu_usage(model_8bit, "8bit")
print_model_gpu_usage(model_4bit, "4bit")

Model: base (GB): 1.324785664
Model: fp16 (GB): 0.662392832
Model: 8bit (GB): 0.359354368
Model: 4bit (GB): 0.207835136


We decreased model memory usage from 1.32GB to 0.2GB ~ 15% of original size. But loading in 8bit and 4bit comes with a bit of performance degradation. Keep in mind that we also need to save some GPU memory for training, so we don't want to take the whole memory just by loading the model.

If we are interested in only running the inference on short sequences, we do not need that much extra memory. Memory requirements also increase if we will be processing very long sequences.

In [5]:
# clear memory
import gc
import torch
del model
del model_fp16
del model_8bit
del model_4bit
gc.collect()
with torch.no_grad():
  torch.cuda.empty_cache()

## Load the model for Fine-Tuning
Now we will load the model that we will be working with in 8bit. If you are working locally, after downloading the model weights for the first time they will be cached, so following model loadings will be much faster.

In [7]:
# Falcon base model for tuning
model_name = "tiiuae/falcon-rw-1b"

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype="auto", load_in_8bit=True)

print_model_gpu_usage(model, "base")

Model: base (GB): 1.41529088


In [9]:
# Print only first 15 layers
for i, (name, param) in enumerate(model.named_parameters()):
    print(f"{name:60s} {str(param.dtype):18s} {str(param.requires_grad):6s}  {param.shape}")
    if i == 15:
        break

transformer.word_embeddings.weight                           torch.bfloat16     True    torch.Size([50304, 2048])
transformer.h.0.self_attention.query_key_value.weight        torch.int8         False   torch.Size([6144, 2048])
transformer.h.0.self_attention.query_key_value.bias          torch.bfloat16     False   torch.Size([6144])
transformer.h.0.self_attention.dense.weight                  torch.int8         False   torch.Size([2048, 2048])
transformer.h.0.self_attention.dense.bias                    torch.bfloat16     False   torch.Size([2048])
transformer.h.0.mlp.dense_h_to_4h.weight                     torch.int8         False   torch.Size([8192, 2048])
transformer.h.0.mlp.dense_h_to_4h.bias                       torch.bfloat16     False   torch.Size([8192])
transformer.h.0.mlp.dense_4h_to_h.weight                     torch.int8         False   torch.Size([2048, 8192])
transformer.h.0.mlp.dense_4h_to_h.bias                       torch.bfloat16     False   torch.Size([2048])
transf

Loading model in 8bit changed attention query, key, value and mlp weight layers in transformer block to int8 and locked them.
Only embedding, layernorm and classification head layers (not visible here) are kept unlocked and loaded in bfloat16.

They are loaded in bfloat16 because we set `torch_dtype="auto"` and in the model configuration file bfloat16 is the type used. You can see the model config file in Hugging Face Hub or write `model.config`

## PEFT - Parameter Efficient Fine Tuning
LLMs are huge! We can't train the whole model for multiple reasons:

*   We probably do not have enough data (overfit issues)
*   Not enough hardware to run it in a reasonable amount of time
*   Very expensive to rent GPUs

Solution is to train only a subset of parameters using techniques like LoRA.

The idea is to freeze all layers and inject additional layers that will be trained. These additional layers are two lower rank matrices which means that they have less parameters than the original weight matrix.

In [10]:
import peft

lora_config = peft.LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=16,           # Rank of update matrices, lower the rank lower number of parameters to train
    lora_alpha=32,  # LoRA scaling factor
    lora_dropout=0.05,
    bias="none",  # Should bias parameter be trained (of the original model); if True this will change the base model biases not just Lora AB matrices
    target_modules=["query_key_value"] # Print model parameters name to figure out which modules to target
)

peft_model = peft.get_peft_model(model, lora_config)  # this changes the model in-place

In [11]:
# Print only first 15 layers
for i, (name, param) in enumerate(model.named_parameters()):  # deliberately using model instead of peft_model
    print(f"{name:75s} {str(param.dtype):18s} {str(param.requires_grad):6s}  {param.shape}")
    if i == 15:
        break

transformer.word_embeddings.weight                                          torch.bfloat16     False   torch.Size([50304, 2048])
transformer.h.0.self_attention.query_key_value.weight                       torch.int8         False   torch.Size([6144, 2048])
transformer.h.0.self_attention.query_key_value.bias                         torch.bfloat16     False   torch.Size([6144])
transformer.h.0.self_attention.query_key_value.lora_A.default.weight        torch.float32      True    torch.Size([16, 2048])
transformer.h.0.self_attention.query_key_value.lora_B.default.weight        torch.float32      True    torch.Size([6144, 16])
transformer.h.0.self_attention.dense.weight                                 torch.int8         False   torch.Size([2048, 2048])
transformer.h.0.self_attention.dense.bias                                   torch.bfloat16     False   torch.Size([2048])
transformer.h.0.mlp.dense_h_to_4h.weight                                    torch.int8         False   torch.Size([8192

We can see how peft added `lora_A` and `lora_B` layers (in float32) and locked all other layers. When multiplied, matrix B*A correspond to `query_key_value.weight` dimension but overall has fewer parameters.

Those are the only layers we will be training.

In [12]:
peft_model.print_trainable_parameters()

trainable params: 3,145,728 || all params: 1,314,770,944 || trainable%: 0.2392605354077554


## Tokenizer
Tokenizer converts text into small chunks - tokens. Each token is associated with an ID. This list of token IDs is the actual input to LLMs.

```
#       Text     --->           Tokens               --->    Input ids
"This is great!" ---> ["This", "_is", "_great", "!"] ---> [52, 345, 124, 13]
````

Note: It is important to use the same tokenizer that was used during model pretraining. In practice, this means loading the tokenizer with the same name as the model.

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [14]:
# Base models are often trained without padding tokens
assert tokenizer.pad_token_id is None

In [15]:
# Often used in various tutorials as a quick PoC fix
# We'll see that this approach is not optimal
tokenizer.pad_token = tokenizer.eos_token

In [16]:
tokenized_text = tokenizer("Large language models are")
print(tokenized_text)

{'input_ids': [21968, 3303, 4981, 389], 'attention_mask': [1, 1, 1, 1]}


When calling the tokenizer directly, it uses the `__call__` method which returns `input_ids` and `attention_mask` directly.

In [17]:
tokenizer.tokenize("Large language models are")

['Large', 'Ġlanguage', 'Ġmodels', 'Ġare']

If you want to see the tokens first, you can use `tokenizer.tokenize` method.

Or you can convert ids to tokens using `tokenizer.convert_ids_to_tokens`.

In [18]:
tokenizer.convert_ids_to_tokens(tokenized_text['input_ids'])

['Large', 'Ġlanguage', 'Ġmodels', 'Ġare']

`attention_mask` is useful for batch processing. Different texts, when tokenized have different number of tokens, but in order to create a batch we need to have the same number of tokens. We can add some token to the end of sequences to make them have the same number of toknes. This token is called padding token. But when we process the text we want to ignore these artificially added tokens so we use `attention_mask`. It tells the attention mechanism which tokens to include and which to ignore during its calculation.

In [19]:
tokenizer(["Large lanugage models are", "I am"], padding=True)

{'input_ids': [[21968, 26992, 1018, 496, 4981, 389], [40, 716, 50256, 50256, 50256, 50256]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 0, 0, 0, 0]]}

First sentece has 6 tokens, while the second one has only 2 tokens. So we need to add 4 padding tokens (with the ID = 50256). In this case they are paded from the right. Attention mask for first sentence contains only 1s because all tokens are relevant, while in the second sentece, last 4 tokens are padded so attention mask is 0 for these tokens. This way attention mechanism will know to look only first 2 tokens as the relevant ones for this example

In order to decode input ids back to text, we can use the `tokenizer.decode` method

In [20]:
tokenizer.decode(tokenized_text['input_ids'])

'Large language models are'

In [21]:
print("Padding token: ", tokenizer.pad_token)
print("Padding token id: ", tokenizer.pad_token_id)

Padding token:  <|endoftext|>
Padding token id:  50256


## Inference

In order to run inference, we need to:
1. Tokenize the text in order to get `input_ids` and `attention_mask`
2. Pass `input_ids` and `attention_mask` to `model.generate` function
3. Decode the output of the model back to text

In [22]:
import torch

text = "What are large language models?"

# 1. Tokenize
model_input = tokenizer(text, return_tensors="pt").to("cuda")  # return input_ids and attention_mask as pytorch tensors ("pt")
print(model_input)

{'input_ids': tensor([[2061,  389, 1588, 3303, 4981,   30]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [23]:
# 2. Generate
peft_model.eval()  # make sure model is in evaluation mode
with torch.no_grad():
    out_tokens = peft_model.generate(
        **model_input,       # unpack input_ids and attention_mask
        max_new_tokens=100,  # How many new tokens to generate
        do_sample=True,      # Greedy search or sampling?
        top_p=0.95,          # Consider only minimal amount of tokens that cover 95% of overall probability
        temperature=0.4,     # Skew the probability -> the lower the number more deterministic
        pad_token_id=tokenizer.pad_token_id
        )

print(out_tokens)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


tensor([[ 2061,   389,  1588,  3303,  4981,    30,   198, 21968,  3303,  4981,
           389,   257,   649,  2099,   286,  3303,  2746,   326,   318,  6007,
           286,  4673,   290, 14607,   546,   257,  1588, 25818,   286,  8950,
            11,   475,   318,   407,  6646,  6007,   286, 15453,   649,  8950,
            13,   198,  2061,   318,   262,  3580,  1022,   257,  3303,  2746,
           290,   257,  3303, 17301,    30,   198,    32,  3303,  2746,   318,
           257,  2446,   329,  4673,   284,  7564,   257,   649,  3303,    13,
           632,   460,  2193,   284,  7564,  2456,   290,  6827,  8573,   286,
           257,  1813,  3303,    11,   290,   788,  4439,   257,   649,  3303,
           326,   318,  2092,   284,   262,  2656,    13,   317,  3303, 17301,
           318,   257,  2891,   326, 18616,   257]], device='cuda:0')


In [24]:
# 3. Convert back to text
print(tokenizer.decode(out_tokens[0]))

What are large language models?
Large language models are a new type of language model that is capable of learning and reasoning about a large vocabulary of languages, but is not necessarily capable of generating new languages.
What is the difference between a language model and a language generator?
A language model is a method for learning to recognize a new language. It can learn to recognize words and sentence structures of a given language, and then produce a new language that is similar to the original. A language generator is a tool that generates a


In [71]:
# Let's put this into function
def generate_response(input, max_new_tokens=100, do_sample=True, top_p=0.95, temperature=0.4):
  batch = tokenizer(input, return_tensors="pt").to("cuda")

  peft_model.eval()
  with torch.no_grad():
    out_tokens = peft_model.generate(
        **batch,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=top_p,
        temperature=temperature,
        pad_token_id=tokenizer.pad_token_id)

  out = tokenizer.decode(out_tokens[0])
  return out

In [72]:
response = generate_response("In order to learn how to draw you need to")
print("\n")
print(response)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




In order to learn how to draw you need to know the basics of drawing and some basic principles. Drawing is an art form that requires a lot of practice and patience. It requires a lot of practice and patience to be able to draw well.
Drawing is a skill that can be learned and improved upon. Once you master the basics of drawing, you will be able to draw anything you want. Drawing is a great way to express yourself and learn new things.
There are many different ways to draw and many different techniques to use when drawing


In [73]:
response = generate_response("Write 5 steps to learn how to draw")
print("\n")
print(response)



Write 5 steps to learn how to draw a car in Illustrator. This step-by-step guide will show you how to draw a car in Illustrator. It includes tips and tricks for drawing realistic looking cars, plus a free car template to download.
This article is part of the How to Draw Cars series. For more tutorials, check out the rest of the series.
How to Draw a Car in Illustrator
Step 1. Draw a circle.
Draw a circle in the center of the page. It should be


## Dataset

To align our model to answer human questions and follow instructions we will fine-tune it on a dataset containing instructions and desired answers.
We will use alpaca, version that has train-validation-test splits.

In [28]:
from datasets import load_dataset

dataset = load_dataset("disham993/alpaca-train-validation-test-split")

train = dataset['train']
val = dataset['validation']
test = dataset['test']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/36401 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7801 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7800 [00:00<?, ? examples/s]

In [29]:
train[0]

{'instruction': 'Generate a story about a girl who discovers a magical item.',
 'input': '',
 'output': 'Once upon a time, there was a young girl named Leila. She was always dreaming of a life of adventure and mystery. One day, while wandering in the woods near her house, she stumbled upon a mysterious item. It was a small box made of a strange material she had never seen before and it was glowing with a magical light. When she opened the box, she was amazed to find a beautiful golden necklace with a glowing stone. She put on the necklace and with a spark of light it vanished into thin air. Suddenly, Leila found herself taken to a far away land, full of mysterious creatures and magical wonders.',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate a story about a girl who discovers a magical item.\n### Response:\nOnce upon a time, there was a

Datasets usually contain instruction/input/question and output/answer etc. In the end model needs one field where everything is combined, like the "text" field here. This "text" field usually follows a specific format that we can create. After fine-tuning, during inference we will get the best results if we follow that format.

We will use the format provided in the "text" field.

Format if input field is not empty:

```
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
```

Format if input field is empty:
```
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
```

## Training

Because we want to fine-tune the model only on the responses, not the whole prompt text, we use a data collator that makes sure to calculate the loss only for the response. That is why we need standardized response format. We tell the collator after which phrase the response starts. Then it uses the tokenizer to convert that phrase into a list of tokens. When that list of tokens is found in the prompt, it knows that is should count the loss only for tokens that come after it. If it can't find that list of tokens in the prompt, it will raise a warning.

In [30]:
from trl import DataCollatorForCompletionOnlyLM, SFTTrainer

response_template = "\n### Response:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

To train the model we use `SFTTrainer` from `trl` library (also in the hugging face ecosystem).

In [31]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir="./results_2023_10",
    evaluation_strategy="steps",
    max_steps=800,
    eval_steps=100,
    logging_steps=100,
    logging_first_step=True,
    save_steps=100,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # Accumulate gradiens to have larger batch size while keeping GPU memory usage low
    learning_rate=5e-5,
    weight_decay=0,
    fp16=True, # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
    group_by_length = True
)

trainer = SFTTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=val.select(indices=range(0, 500)),  # Let's take a subset of val so that evaluation is faster
    dataset_text_field="text",  # Field that contains the final prompt to use
    data_collator=collator,     # Send our collator that will calculate loss only for response
    args=training_arguments,
    max_seq_length=384
)

Map:   0%|          | 0/36401 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [32]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.485200,1.518560
200,1.495900,1.489727
300,1.461400,1.478224
400,1.423800,1.470058
500,1.420500,1.464101
600,1.402500,1.458479
700,1.399000,1.455378
800,1.405900,1.452707
900,1.365800,1.451079
1000,1.369100,1.450881


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

TrainOutput(global_step=1000, training_loss=1.4231065338850022, metrics={'train_runtime': 2319.8998, 'train_samples_per_second': 3.448, 'train_steps_per_second': 0.431, 'total_flos': 6662749843537920.0, 'train_loss': 1.4231065338850022, 'epoch': 0.22})

In [33]:
# Save trained model
trainer.model.save_pretrained("falcon-rw-1b-alpaca")

In [60]:
INFERENCE_TEMPLATE = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n### Response:"

instruction = "Write 5 steps to learn how to draw"
inference_input = INFERENCE_TEMPLATE.format(instruction=instruction)

print(inference_input)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write 5 steps to learn how to draw
### Response:


In [66]:
# Much better than before fine-tuning, but it still has a hard time stopping when done.
response = generate_response(inference_input, max_new_tokens=150, do_sample=True)
print("\n")
print(response)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write 5 steps to learn how to draw
### Response:
Step 1: Gather the necessary materials 
Step 2: Draw an outline of the object to be drawn 
Step 3: Add features and details 
Step 4: Color the drawing 
Step 5: Evaluate the drawing 
This task involves gathering the necessary materials, drawing an outline of the object to be drawn, adding features and details, coloring the drawing, and evaluating the drawing. The steps can be practiced individually or in a group setting. For example, a group could practice drawing an object, drawing an outline of the object, adding features and details, coloring the drawing, and evaluating the drawing. This task can be done individually as well, but the group setting can provide a more interactive experience.<|endoftext|>A good


In [67]:
# Better
response = generate_response(inference_input, max_new_tokens=150, do_sample=False)
print("\n")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write 5 steps to learn how to draw
### Response:
1. Learn the basics of drawing. 
2. Practice drawing with a variety of materials. 
3. Learn to use different techniques and styles. 
4. Learn to use perspective and perspective drawing. 
5. Learn to use color and shading. 

#


## Merge LoRA weights

In [68]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = peft.PeftModel.from_pretrained(base_model, "falcon-rw-1b-alpaca")
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [74]:
def generate_response(input, max_new_tokens=100, do_sample=True, top_p=0.95, temperature=0.4):
  batch = tokenizer(input, return_tensors="pt").to("cuda")

  model.eval()
  with torch.no_grad():
    out_tokens = model.generate(
        **batch,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=top_p,
        temperature=temperature,
        pad_token_id=tokenizer.pad_token_id)

  out = tokenizer.decode(out_tokens[0])
  return out

In [79]:
response = generate_response(inference_input, max_new_tokens=150, do_sample=True, temperature=0.4)
print("\n")
print(response)



Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write 5 steps to learn how to draw
### Response:
1. Learn the basics of drawing. 
2. Practice drawing regularly. 
3. Draw with different materials. 
4. Learn to use different techniques. 
5. Learn to master different styles. 

#


## Push to Hub

In [80]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [81]:
model.push_to_hub("falcon-rw-1b-alpaca", use_temp_dir=False)
tokenizer.push_to_hub("falcon-rw-1b-alpaca", use_temp_dir=False)

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mfernezir/falcon-rw-1b-alpaca/commit/6a2548166b77ba9507b1d17d26dd21185a947de9', commit_message='Upload tokenizer', commit_description='', oid='6a2548166b77ba9507b1d17d26dd21185a947de9', pr_url=None, pr_revision=None, pr_num=None)

## Homework exercises

- Try different inference and fine-tuning parameters
- Use separate EOS, BOS, and Padding tokens
- Fine-tune another model, such as LLaMA 2. Reference: https://mlabonne.github.io/blog/posts/Fine_Tune_Your_Own_Llama_2_Model_in_a_Colab_Notebook.html

Currently, by using EOS token as padding, the model never learns to predict the EOS token (model never predicts padding tokens since they are masked)
